In [3]:
# Import usual libraries
import numpy as np
import matplotlib.pyplot as plt
import itertools


import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import numpy as np

# 1 - DATA MANIPULATION
import pandas as pd
import numpy as np

# 2 - DATA VISUALISATION
import matplotlib.pyplot as plt
import seaborn as sns

# 3 - STATISTICS
from statsmodels.graphics.gofplots import qqplot

# 4 - MACHINE LEARNING
## 4.1 - Preprocessing

### 4.1.1 - Scalers
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

### 4.1.2 - Encoders
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

### 4.1.3 - Crossvalidation, Training, Model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor

### 4.1.4 - Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics  import ConfusionMatrixDisplay

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from math import sqrt

# Make all figures tiny for readability purpose
from matplotlib import rcParams
rcParams['figure.figsize'] = (5,3)

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd


In [16]:
gdpnow = pd.read_csv('gdpnow_hf.csv', index_col='Dates', parse_dates=True) #date_parser=dateparse)
# Ensure that load_df index is in the same date format
gdpnow.index = pd.to_datetime(gdpnow.index)

In [17]:
Target = 'Final_GDP_Interp'
Drop = ['GDP Nowcast', 'Final_GDP_Interp', 'Quarter being forecasted', 'Advance Estimate From BEA', 'Publication Date of Advance Estimate',
       'Days until advance estimate', 'Forecast Error', 'Data releases']

In [18]:

X = gdpnow.drop(columns=Drop)
y = gdpnow[Target]

y= y[-X.shape[0]:]

In [19]:
# y = y.fillna(method='ffill')
# y = pd.Series(y)
y

Dates
2014-05-01    0.110000
2014-05-02    0.152667
2014-05-05    0.280667
2014-05-06    0.323333
2014-05-07    0.366000
                ...   
2024-01-15    3.374353
2024-01-16    3.355765
2024-01-17    3.337176
2024-01-18    3.318588
2024-01-19    3.300000
Name: Final_GDP_Interp, Length: 2537, dtype: float64

In [20]:
from sklearn.model_selection import train_test_split

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


In [50]:
## Non Linear PCA
# from sklearn.decomposition import KernelPCA
# from sklearn.pipeline import make_pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler

# # Updated preprocessing pipeline with Kernel PCA
# preproc = make_pipeline(
#     SimpleImputer(strategy='mean'),
#     StandardScaler(),
#     KernelPCA(n_components=None, kernel='rbf')  # Example with RBF kernel do not enter the %
# )

# PCA Adjusted preprocessing pipeline with named steps
# preproc = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler()),
#     ('pca', PCA(n_components=0.95, random_state=42))  # Named step 'pca'
# ])

In [21]:

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, mutual_info_regression
from sklearn.decomposition import PCA

# Adjusted preprocessing pipeline with named steps
preproc = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=3)),
    ('scaler', RobustScaler()),

])

preproc_selector = Pipeline([
    ('preprocessing', preproc),  # Include the preprocessing steps with PCA
    ('feature_selection', SelectPercentile(
        mutual_info_regression,
        percentile=90 # Keep 90% of all features
    ))
])

In [52]:

model = XGBRegressor(random_state=42)

param_distributions = {
    'model__n_estimators': [200],
    'model__learning_rate': [0.1],
    'model__max_depth': [16, 20],
}

pipe = Pipeline([
    ('preprocessor', preproc),
    ('model', model)
])

random_search = RandomizedSearchCV(
    pipe,
    param_distributions=param_distributions,
    n_iter=100,  # Number of parameter settings that are sampled. n_iter trades off runtime vs quality of the solution.
    scoring='neg_mean_absolute_error',  # Assuming MSE is the metric of interest; adjust as needed.
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available cores
)

random_search.fit(X_train, y_train)

print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", random_search.best_score_)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


/Users/thomasbowden/.pyenv/versions/lewagon/envs/project-btm/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 2 is smaller than n_iter=100. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[CV] END model__learning_rate=0.1, model__max_depth=20, model__n_estimators=200; total time=   3.0s
[CV] END model__learning_rate=0.1, model__max_depth=20, model__n_estimators=200; total time=   3.1s
[CV] END model__learning_rate=0.1, model__max_depth=16, model__n_estimators=200; total time=   3.1s
[CV] END model__learning_rate=0.1, model__max_depth=16, model__n_estimators=200; total time=   3.0s
[CV] END model__learning_rate=0.1, model__max_depth=16, model__n_estimators=200; total time=   3.2s
[CV] END model__learning_rate=0.1, model__max_depth=16, model__n_estimators=200; total time=   3.2s
[CV] END model__learning_rate=0.1, model__max_depth=16, model__n_estimators=200; total time=   3.3s
[CV] END model__learning_rate=0.1, model__max_depth=20, model__n_estimators=200; total time=   3.4s
[CV] END model__learning_rate=0.1, model__max_depth=20, model__n_estimators=200; total time=   1.5s
[CV] END model__learning_rate=0.1, model__max_depth=20, model__n_estimators=200; total time=   1.5s


In [22]:
# genrally higher N_estimators and lower learning rates are more robust
# Best parameters found:  {'model__n_estimators': 150, 'model__max_depth': 6, 'model__learning_rate': 0.1}
# Best score found:  -0.10133019738065827

# Defining the best model
model_best =  XGBRegressor(n_estimators=200, max_depth = 16, learning_rate = 0.1, random_state=42)

# Creating the pipeline with memory caching
pipe_best = make_pipeline(preproc_selector, model_best)

# Assuming X_train and y_train are already defined and contain your training data
score = cross_val_score(pipe_best, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')

# Printing the standard deviation and mean of the cross-validation scores
print(score.std())
print(score.mean())

0.05381251060833955
-0.49307324150226267


In [23]:
pipe_best.fit(X_train,y_train)
y_pred = pd.Series(pipe_best.predict(X_test)).reset_index(drop=True)
y_test = pd.Series(y_test).reset_index(drop=True)

In [24]:

from statsmodels.tsa.stattools import acf

def forecast_accuracy(y_pred: pd.Series, y_true: pd.Series) -> float:

    mape = np.mean(np.abs(y_pred - y_true)/np.abs(y_true))  # Mean Absolute Percentage Error
    me = np.mean(y_pred - y_true)             # ME
    mae = np.mean(np.abs(y_pred - y_true))    # MAE
    mpe = np.mean((y_pred - y_true)/y_true)   # MPE
    rmse = np.mean((y_pred - y_true)**2)**.5  # RMSE
    corr = np.corrcoef(y_pred, y_true)[0,1]   # Correlation between the Actual and the Forecast
    mins = np.amin(np.hstack([y_pred.values.reshape(-1,1), y_true.values.reshape(-1,1)]), axis=1)
    maxs = np.amax(np.hstack([y_pred.values.reshape(-1,1), y_true.values.reshape(-1,1)]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(y_pred-y_true, fft=False)[1]                      # Lag 1 Autocorrelation of Error

    forecast = ({
        'mape':mape,
        'me':me,
        'mae': mae,
        'mpe': mpe,
        'rmse':rmse,
        'acf1':acf1,
        'corr':corr,
        'minmax':minmax
    })

    return forecast


In [25]:
forecast_accuracy(y_pred,y_test)

{'mape': 0.2545291284531343,
 'me': -0.021508168415027982,
 'mae': 0.4442398624232382,
 'mpe': 0.13265314768656827,
 'rmse': 1.6760666343606618,
 'acf1': 0.03456937139116738,
 'corr': 0.9639912498354158,
 'minmax': 0.0665763868170659}

In [26]:
# Baseline no additional data
{'mape': 0.4963433333215978,
 'me': 0.1525253749298041,
 'mae': 1.0733358723767286,
 'mpe': 0.04441432676896753,
 'rmse': 3.325099172055762,
 'acf1': 0.019003663322351982,
 'corr': 0.7977906473665369,
 'minmax': 0.21016413941493906}

# Baseline with High Frequency
{'mape': 0.2545291284531343,
 'me': -0.021508168415027982,
 'mae': 0.4442398624232382,
 'mpe': 0.13265314768656827,
 'rmse': 1.6760666343606618,
 'acf1': 0.03456937139116738,
 'corr': 0.9639912498354158,
 'minmax': 0.0665763868170659}

# GDP Now
{'mape': 1.051006638571981,
 'me': 0.417513745814221,
 'mae': 2.6060549265876,
 'mpe': 0.36004739951206266,
 'rmse': 6.944290781867663,
 'acf1': 0.9898463580140768,
 'corr': 0.5758029675582143,}


{'mape': 1.051006638571981,
 'me': 0.417513745814221,
 'mae': 2.6060549265876,
 'mpe': 0.36004739951206266,
 'rmse': 6.944290781867663,
 'acf1': 0.9898463580140768,
 'corr': 0.5758029675582143}

In [27]:
model_forecast = pd.DataFrame()

# Assign rounded values of y_pred and y_test to the DataFrame
model_forecast['y_pred'] = y_pred.values
model_forecast['y_test'] = y_test.values

# Display the DataFrame
model_forecast

,y_pred,y_test
0,1.286994,1.268462
1,2.931285,3.376240
2,6.437019,6.440769
3,0.840852,0.776957
4,2.432255,2.478876
...,...,...
503,2.016037,2.035208
504,3.941055,5.712637
505,3.131536,0.779011
506,0.727237,0.820435


Viewing the PCA

In [58]:
# Assuming X_train and y_train are your features and target variables respectively
preproc.fit(X_train, y_train)


Pipeline(steps=[('imputer', KNNImputer()), ('scaler', RobustScaler())])

In [59]:
# Fit the preprocessing pipeline and transform X_train
X_train_transformed = preproc.fit_transform(X_train)

In [60]:
import pandas as pd

# Convert the transformed array into a DataFrame
transformed_df = pd.DataFrame(X_train_transformed,
                              columns=[f'Feature_{i}' for i in range(X_train_transformed.shape[1])])

# Display the transformed DataFrame
print(transformed_df)


      Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
1     -3.213498  -1.212457   6.800304   0.516424  -2.526959  -1.354394   
2      0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
3      5.882307   8.087543  -3.088585  -7.704362  15.273041   5.077121   
4     -1.140190   1.687543   7.578082   0.709855  -3.046959   2.837576   
...         ...        ...        ...        ...        ...        ...   
2024   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
2025   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
2026 -18.395467   1.487543   8.244748  -2.675175 -19.566959 -10.599696   
2027   6.751759  12.037543  -7.755252  10.478083   9.313041   4.215757   
2028  -0.337618   1.937543   1.466970   0.468067   0.273041   0.138636   

      Feature_6  Feature_7  Feature_8  Feature_9  ...  Feature_18  Feature_19  \
0      0.000000   0.000000   0

In [61]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import shap
from xgboost import XGBRegressor

# Fit your model on the transformed data
model = XGBRegressor(n_estimators=200, max_depth=20, learning_rate=0.1, random_state=42)
model.fit(X_train_transformed, y_train)

# Compute SHAP values to identify influential components
explainer = shap.Explainer(model)
shap_values = explainer(X_train_transformed)

# Assuming we're interested in the top 1 component for demonstration
top_components = np.argsort(-np.sum(np.abs(shap_values.values), axis=0))[0:1]

# Extract PCA step from the pipeline
pca = preproc.named_steps['pca']

# Extract PCA loadings for the top influential components
loadings = pca.components_[top_components]

# Assuming X_train is a DataFrame and has column names
feature_names = X_train.columns

# Create a DataFrame for loadings
loading_df = pd.DataFrame(loadings, columns=feature_names, index=[f'Component {i+1}' for i in top_components])

print("PCA Loadings for the top influential components:")
print(loading_df)

ModuleNotFoundError: No module named 'shap'

In [ ]:
# Assuming loading_df is already created and contains the PCA loadings for Component 1
# Extract loadings for Component 1
component_1_loadings = loading_df.loc['Component 1']

# Sort the loadings by their absolute values in descending order to find the top features
sorted_loadings = component_1_loadings.abs().sort_values(ascending=False)

# Select the top 20 features based on their loadings
top_50_features = sorted_loadings.head(50)

# Convert the series to a list of feature names
top_50_feature_names = top_50_features.index.tolist()

# Optionally, to get the corresponding loading values as well
top_50_feature_values = top_50_features.values.tolist()

print("Top 50 PCA Loadings for Component 1 (Features):", top_50_feature_names)
print("Top 50 PCA Loadings for Component 1 (Values):", top_50_feature_values)

In [ ]:
gdpnow_test = gdpnow['GDP Nowcast']
gdpnow_test = gdpnow_test.interpolate(method='time')
gdp_test = gdpnow['Final_GDP_Interp']


In [ ]:
gdpnow_test = pd.Series(gdpnow_test).reset_index(drop=True)
gdp_test = pd.Series(gdp_test).reset_index(drop=True)

In [ ]:
forecast_accuracy(gdpnow_test,gdp_test)

/var/folders/v5/5q966zvj59j8p5z9qllhcr7r0000gp/T/ipykernel_55229/2581306049.py:13: RuntimeWarning: divide by zero encountered in divide
  minmax = 1 - np.mean(mins/maxs)             # minmax


{'mape': 1.051006638571981,
 'me': 0.417513745814221,
 'mae': 2.6060549265876,
 'mpe': 0.36004739951206266,
 'rmse': 6.944290781867663,
 'acf1': 0.9898463580140768,
 'corr': 0.5758029675582143,
 'minmax': inf}

In [ ]:
{'mape': 0.3793836113654958,
 'me': 0.050791386252946914,
 'mae': 1.2648437748039127,
 'mpe': 0.01996571667170425,
 'rmse': 4.054536843585369,
 'acf1': -0.01184796160451125,
 'corr': 0.7459617866029101,
 'minmax': 0.354164318228377}

{'mape': 0.3793836113654958,
 'me': 0.050791386252946914,
 'mae': 1.2648437748039127,
 'mpe': 0.01996571667170425,
 'rmse': 4.054536843585369,
 'acf1': -0.01184796160451125,
 'corr': 0.7459617866029101,
 'minmax': 0.354164318228377}

In [ ]:
{'mape': 1.051006638571981,
 'me': 0.417513745814221,
 'mae': 2.6060549265876,
 'mpe': 0.36004739951206266,
 'rmse': 6.944290781867663,
 'acf1': 0.9898463580140768,
 'corr': 0.5758029675582143,
 'minmax': inf}